In [1]:
from Framework_V2.core.config import CaseConfig, GridShape
from Framework_V2.core.geometry import FinnTubedHX
from Framework_V2.models import Frostmodell_V1
from Framework_V2.runtime.simulator import Simulator
from Framework_V2.visualisation import plot
from CoolProp.HumidAirProp import HAPropsSI

# Case Definieren

In [2]:
T_a = 16
T_w = -10
P = 103500
RH = 0.8
v_a = 1

cfg = CaseConfig(
    # air data
    T_a = T_a,          # °C temperature air
    v_a = v_a,          # m/s velocity air
    p_a = P,            # Pa pressure air
    RH = RH,            # relative humidity air
    w_amb = HAPropsSI('W','T',T_a+273.15,'P',P,'R', RH),          # kg/kg water vapor moisture content
    rho_amb = 1.2,      # kg/m^3 density air
    v_kin = 1.5e-5,     # m^2/s kinematic viscosity air
    lam = 0.025,        # W/mK heat conduction coefficient air
    c_p_a = 1000,       # J/kgK heat capacity air
    D_std = 2.2e-5,     # m^2/s water vapor diffusion coefficient
    C = 900,            # 1/s empirical water vapor absorbed coefficient
    isv = 2830000,      # J/kg latent heat of desublimation

    # refrigerant data
    T_w = T_w,          # °C wall temperature

    # ice data
    rho_i = 920,        # kg/m^3 frost density (solid)
    h_sub= 334,         # kJ/kg latent heat of ablimation for water vapor

    # numerics
    t_end = 60*60,      # s endtime
    dt = 5              # s time step
)

geom = FinnTubedHX(
    n_fin = 4,           # -
    l_fin = 0.01,          # m
    d_fin = 0.002,    # m
    fin_pitch = 0.01,          # m
    d_tube_a = 0.01,          # m
    tube_thickness = 0.002       # m
)

gs = GridShape(
    nx = 100,
    nr = 100,
    ntheta = 1
)

# Simulation laufen lassen

In [3]:
sim = Simulator(fields=("t","s_e"))
results = sim.run(cfg, geom, gs, Frostmodell_V1)

Time Step: 0 s 	 Inner Iterations: 4 	 w: 4.854e-09 	 T: 5.473e-08
Time Step: 1 s 	 Inner Iterations: 1 	 w: 6.644e-10 	 T: 8.481e-01
Time Step: 2 s 	 Inner Iterations: 2 	 w: 9.755e-07 	 T: 2.568e-05
Time Step: 3 s 	 Inner Iterations: 1 	 w: 4.355e-07 	 T: 1.279e-02
Time Step: 4 s 	 Inner Iterations: 2 	 w: 1.261e-07 	 T: 1.655e-05
Time Step: 5 s 	 Inner Iterations: 1 	 w: 4.001e-08 	 T: 9.482e-04
Time Step: 6 s 	 Inner Iterations: 1 	 w: 4.573e-07 	 T: 5.287e-05
Time Step: 7 s 	 Inner Iterations: 1 	 w: 3.097e-07 	 T: 2.953e-04
Time Step: 8 s 	 Inner Iterations: 1 	 w: 3.508e-07 	 T: 1.829e-04
Time Step: 9 s 	 Inner Iterations: 1 	 w: 1.998e-07 	 T: 2.510e-04
Time Step: 10 s 	 Inner Iterations: 1 	 w: 1.709e-07 	 T: 1.827e-04
Time Step: 11 s 	 Inner Iterations: 1 	 w: 1.283e-07 	 T: 1.823e-04
Time Step: 12 s 	 Inner Iterations: 1 	 w: 9.841e-08 	 T: 1.599e-04
Time Step: 13 s 	 Inner Iterations: 1 	 w: 6.470e-08 	 T: 1.467e-04
Time Step: 14 s 	 Inner Iterations: 1 	 w: 4.875e-08 	 T: 

ValueError: The input for key (12) with value (127.07) is outside the range of validity: (130) to (623.15) :: inputs were:"W","T",1.2706999186761630e+02,"P",1.0350000000000000e+05,"R",1.0000000000000000e+00 

# Resultate speichen

In [ ]:
result_file = "results_test_edge.csv"
results.to_csv(result_file)

# Resultate plotten

In [ ]:
plot_r = 99
plot_theta = 0
plot_time = int(430 / cfg.dt)

plot.plot_xy(x=results.data['t'],y=results.data['s_e'],
             xlabel="Zeit [s]", ylabel="Frostdicke [m]",
             title=f"Frostdicke bei θ_idx = {plot_theta}",
             theta_idx=plot_theta, marker=None)
plot.plot_xy(x=results.data['t'],y=results.data['T_e'],
             xlabel="Zeit [s]", ylabel="Temperatur [°C]",
             title=f"Frosttemperatur bei r_ids = {plot_r} und θ_idx = {plot_theta}",
             r_idx=plot_r, theta_idx=plot_theta, marker=None)
plot.plot_xy(x=results.data['t'],y=results.data['rho_e'],
             xlabel="Zeit [s]", ylabel="Dichte [kg/m^3]",
             title=f"Frostdichte bei r_ids = {plot_r} und θ_idx = {plot_theta}",
             r_idx=plot_r, theta_idx=plot_theta, marker=None)
plot.plot_xy(x=results.data['t'],y=results.data['w_e'],
             xlabel="Zeit [s]", ylabel="w [kg/kg]",
             title=f"Feuchtigkeit bei r_ids = {plot_r} und θ_idx = {plot_theta}",
             r_idx=plot_r, theta_idx=plot_theta, marker=None)

In [ ]:
plot.plot_spatial_slice(
    results.data["T_e"], vary="r", t_idx=plot_time, theta_idx=plot_theta,
    title=f"T_e(r) bei t={results.data['t'][plot_time]} s, θ_idx = {plot_theta}", xlabel="r_idx", ylabel="T_e [°C]", marker=None
)
plot.plot_spatial_slice(
    results.data["rho_e"], vary="r", t_idx=plot_time, theta_idx=plot_theta,
    title=f"rho_e(r) bei t={results.data['t'][plot_time]} s, θ_idx = {plot_theta}", xlabel="r_idx", ylabel="rho_e [kg/m^3]", marker=None
)
plot.plot_spatial_slice(
    results.data["w_e"], vary="r", t_idx=plot_time, theta_idx=plot_theta,
    title=f"w_e(r) bei t={results.data['t'][plot_time]} s, θ_idx = {plot_theta}", xlabel="r_idx", ylabel="w_e [kg/kg]", marker=None
)

In [ ]:
plot.plot_spatial_slice(
    y=results.data["s_e"],     # shape: (time, theta)
    vary="theta",                 # we’re slicing along θ
    r_idx=plot_time,
    t_idx=plot_time,                 # pick the time index
    ylabel="s_e [m]",
    title=f"s_e(θ) bei t={results.data['t'][plot_time]} s" if "t" in results.data else "s_e(θ)",
    marker=None
)